**KAGGLE 2025 Pitch by Pitch Dataset**

In [7]:
#gather data
import pandas as pd
mlb_df = pd.read_csv('Data_MLB_2025_StatcastPostseason_PitchByPitch_20251102a.csv')
mlb_df.head()

,game_year,game_date,player_name,pitcher,batter,balls,strikes,inning,inning_topbot,at_bat_number,...,post_home_score,post_bat_score,post_fld_score,if_fielding_alignment,of_fielding_alignment,spin_axis,delta_home_win_exp,delta_run_exp,bat_speed,swing_length
0,2025,2025-09-30,"Ashcraft, Graham",668933,605141,0,0,8,Bot,73,...,10,10,5,Standard,Standard,42.0,0.0,0.041,NaN,NaN
1,2025,2025-09-30,"Ashcraft, Graham",668933,605141,1,0,8,Bot,73,...,10,10,5,Standard,Standard,201.0,0.0,-0.050,NaN,NaN
2,2025,2025-09-30,"Ashcraft, Graham",668933,605141,1,1,8,Bot,73,...,10,10,5,Standard,Standard,23.0,0.0,-0.061,65.0,6.0
3,2025,2025-09-30,"Ashcraft, Graham",668933,605141,1,2,8,Bot,73,...,10,10,5,Standard,Standard,39.0,0.0,0.000,53.1,7.2
4,2025,2025-09-30,"Ashcraft, Graham",668933,605141,1,2,8,Bot,73,...,10,10,5,Standard,Standard,201.0,0.0,0.043,NaN,NaN


In [8]:
print(mlb_df.columns)

Index(['game_year', 'game_date', 'player_name', 'pitcher', 'batter', 'balls',
       'strikes', 'inning', 'inning_topbot', 'at_bat_number', 'pitch_number',
       'events', 'pitch_type', 'pitch_name', 'release_speed', 'release_pos_x',
       'release_pos_z', 'description', 'spin_dir', 'spin_rate_deprecated',
       'break_angle_deprecated', 'break_length_deprecated', 'zone', 'des',
       'game_type', 'stand', 'p_throws', 'home_team', 'away_team', 'type',
       'hit_location', 'bb_type', 'pfx_x', 'pfx_z', 'plate_x', 'plate_z',
       'on_3b', 'on_2b', 'on_1b', 'outs_when_up', 'hc_x', 'hc_y',
       'tfs_deprecated', 'tfs_zulu_deprecated', 'fielder_2', 'fielder_2.1',
       'umpire', 'sv_id', 'vx0', 'vy0', 'vz0', 'ax', 'ay', 'az', 'sz_top',
       'sz_bot', 'hit_distance_sc', 'launch_speed', 'launch_angle',
       'effective_speed', 'release_spin_rate', 'release_extension', 'game_pk',
       'fielder_2.2', 'fielder_2.3', 'fielder_3', 'fielder_4', 'fielder_5',
       'fielder_6', 'field

**DATA CLEANING AND DATABASE NORMALIZATION** 

In [ ]:
# Team df (create team_id?)
#team = pd.read_csv()


# Player df (already have pitcher numbers and batter numbers I believe)
#player = pd.read_csv()

# Game df (create your own game_id?)
#game = pd.read_csv()

**Analysis Questions**

1. Batters on by pitch type category
2. Pitchers on batter left/right handed
3. Batters overall info
4. Pitchers overall info
5. Batters hit locations
6. MOMENTUM analysis (this one should be fun)


-- with more time

1. Salary analysis
2. Age analysis
3. Veteran analysis
4. Overall 2025 comparative stats to postseason